## 1 combine_data

In [1]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

In [2]:
train_path = 'C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\raw\\train.csv'
test_path = 'C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\raw\\test.csv'

In [3]:
train_df = pd.read_csv(train_path, index_col=0)
test_df = pd.read_csv(test_path, index_col=0)

In [4]:
train_df.shape, test_df.shape

((6261, 7), (2504, 6))

In [5]:
train_df["is_train"] = 1
test_df["is_train"] = 0

In [6]:
combine_data = pd.concat([train_df, test_df], sort=False)

In [7]:
combine_data.head(2)

,id,atm_group,address,address_rus,lat,long,target,is_train
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.01150,1
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.02971,1


In [8]:
combine_data.shape

(8765, 8)

In [9]:
combine_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8765 entries, 0 to 2503
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           8765 non-null   float64
 1   atm_group    8765 non-null   float64
 2   address      8765 non-null   object 
 3   address_rus  8345 non-null   object 
 4   lat          8345 non-null   float64
 5   long         8345 non-null   float64
 6   target       6261 non-null   float64
 7   is_train     8765 non-null   int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 616.3+ KB


In [10]:
combine_data.isnull().sum()

id                0
atm_group         0
address           0
address_rus     420
lat             420
long            420
target         2504
is_train          0
dtype: int64

## 2 find_gaps

In [11]:
find_gaps = combine_data[combine_data['address_rus'].isnull()]

In [12]:
find_gaps

,id,atm_group,address,address_rus,lat,long,target,is_train
40,4510.0,496.5,PER. BAZARNY 4 SPASSK-DALNY,NaN,NaN,NaN,0.039191,1
150,347.0,1022.0,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,NaN,NaN,NaN,-0.145001,1
182,1591.0,1942.0,16A BEREJKOVSK NAB MOSCOW,NaN,NaN,NaN,0.080034,1
192,440.0,1942.0,113 MIRA AV. TOGLIATTI,NaN,NaN,NaN,-0.082956,1
214,556.0,1942.0,133A MOZHAYSKOE H/W. MOSCOW,NaN,NaN,NaN,0.061517,1
...,...,...,...,...,...,...,...,...
2346,3199.0,8083.0,"MOROZOVA P.L., 86 KHabarovsk",NaN,NaN,NaN,NaN,0
2399,5354.0,5478.0,"D. 15, UL. 78 DOBROVOL'CH KRASNOYARSK",NaN,NaN,NaN,NaN,0
2471,3333.0,8083.0,"OKTYABRSKAYA, 105 Tver",NaN,NaN,NaN,NaN,0
2474,2247.0,3185.5,"UL. TELEVIZORNAJA, KRASNOJARSK",NaN,NaN,NaN,NaN,0


## 3 gaps_geocoding_via_API

In [13]:
gaps_geocoding = pd.read_csv('C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\external\\geocoded_data.csv')

In [14]:
gaps_geocoding

,Unnamed: 0,source,result,postal_code,country,country_iso_code,federal_district,region_fias_id,region_kladr_id,region_iso_code,...,geo_lat,geo_lon,beltway_hit,beltway_distance,qc_geo,qc_complete,qc_house,qc,unparsed_parts,metro
0,0,PER. BAZARNY 4 SPASSK-DALNY,"Приморский край, г Спасск-Дальний, пер Базарны...",692245.0,Россия,RU,Дальневосточный,43909681-d6e1-432d-b61f-ddac393cb5da,2.500000e+12,RU-PRI,...,44.590117,132.815729,NaN,NaN,4,9,2,1,ДАЛЬНЫ,NaN
1,0,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,"Респ Татарстан, г Набережные Челны",423800.0,Россия,RU,Приволжский,0c089b04-099e-4e0e-955a-6bf1ce525f1a,1.600000e+12,RU-TA,...,55.743462,52.395916,NaN,NaN,4,3,10,1,"АББ 6Б, ЧЕЛЬНИНСКИЙ НАБ",NaN
2,0,16A BEREJKOVSK NAB MOSCOW,г Москва,101000.0,Россия,RU,Центральный,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,7.700000e+12,RU-MOW,...,55.754047,37.620405,NaN,NaN,4,3,10,1,16А БЕРЕЙКОВСК НАБ,NaN
3,0,113 MIRA AV. TOGLIATTI,"Самарская обл, г Тольятти, ул Мира, влд 113",445054.0,Россия,RU,Приволжский,df3d7359-afa9-4aaa-8ff9-197e73906b1c,6.300000e+12,RU-SAM,...,53.510784,49.444468,NaN,NaN,0,5,2,0,NaN,NaN
4,0,133A MOZHAYSKOE H/W. MOSCOW,"г Москва, ш Можайское, д 133А",143026.0,Россия,RU,Центральный,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,7.700000e+12,RU-MOW,...,55.718393,37.415154,IN_MKAD,NaN,2,9,10,1,ДОМ/В,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,0,"MOROZOVA P.L., 86 KHabarovsk","г Хабаровск, ул Морозова Павла Леонтьевича, д 86",680012.0,Россия,RU,Дальневосточный,7d468b39-1afa-41ec-8c4f-97a8603cb3d4,2.700000e+12,RU-KHA,...,48.428996,135.103956,NaN,NaN,0,5,2,0,NaN,NaN
416,0,"D. 15, UL. 78 DOBROVOL'CH KRASNOYARSK",г Красноярск,660000.0,Россия,RU,Сибирский,db9c4f8b-b706-40e2-b2b4-d31b98dcd3d1,2.400000e+12,RU-KYA,...,56.009388,92.852481,NaN,NaN,4,3,10,1,"Д. 15, УЛ. 78 ДОБРОВОЛЬЧ",NaN
417,0,"OKTYABRSKAYA, 105 Tver",г Тверь,170000.0,Россия,RU,Центральный,61723327-1c20-42fe-8dfa-402638d9b396,6.900000e+12,RU-TVE,...,56.858606,35.911676,NaN,NaN,4,3,10,1,"ОКТЯБРСКАЯ, 105",NaN
418,0,"UL. TELEVIZORNAJA, KRASNOJARSK","г Красноярск, ул Телевизорная",NaN,Россия,RU,Сибирский,db9c4f8b-b706-40e2-b2b4-d31b98dcd3d1,2.400000e+12,RU-KYA,...,56.029485,92.792106,NaN,NaN,2,4,10,0,NaN,NaN


## 4 gaps_filling

In [15]:
combine_data.head(2)

,id,atm_group,address,address_rus,lat,long,target,is_train
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.01150,1
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.02971,1


In [16]:
gaps_geocoding.head(2)

,Unnamed: 0,source,result,postal_code,country,country_iso_code,federal_district,region_fias_id,region_kladr_id,region_iso_code,...,geo_lat,geo_lon,beltway_hit,beltway_distance,qc_geo,qc_complete,qc_house,qc,unparsed_parts,metro
0,0,PER. BAZARNY 4 SPASSK-DALNY,"Приморский край, г Спасск-Дальний, пер Базарны...",692245.0,Россия,RU,Дальневосточный,43909681-d6e1-432d-b61f-ddac393cb5da,2.500000e+12,RU-PRI,...,44.590117,132.815729,NaN,NaN,4,9,2,1,ДАЛЬНЫ,NaN
1,0,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,"Респ Татарстан, г Набережные Челны",423800.0,Россия,RU,Приволжский,0c089b04-099e-4e0e-955a-6bf1ce525f1a,1.600000e+12,RU-TA,...,55.743462,52.395916,NaN,NaN,4,3,10,1,"АББ 6Б, ЧЕЛЬНИНСКИЙ НАБ",NaN


In [17]:
gaps_geocoding = gaps_geocoding[gaps_geocoding['qc_geo'] < 2]

In [18]:
gaps_geocoding_unq = gaps_geocoding.drop_duplicates(subset=['source'])

In [19]:
pd.merge(find_gaps, gaps_geocoding_unq,
             how='left',
             left_on='address', 
             right_on='source')

,id,atm_group,address,address_rus,lat,long,target,is_train,Unnamed: 0,source,...,geo_lat,geo_lon,beltway_hit,beltway_distance,qc_geo,qc_complete,qc_house,qc,unparsed_parts,metro
0,4510.0,496.5,PER. BAZARNY 4 SPASSK-DALNY,NaN,NaN,NaN,0.039191,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,347.0,1022.0,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,NaN,NaN,NaN,-0.145001,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1591.0,1942.0,16A BEREJKOVSK NAB MOSCOW,NaN,NaN,NaN,0.080034,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,440.0,1942.0,113 MIRA AV. TOGLIATTI,NaN,NaN,NaN,-0.082956,1,0.0,113 MIRA AV. TOGLIATTI,...,53.510784,49.444468,NaN,NaN,0.0,5.0,2.0,0.0,NaN,NaN
4,556.0,1942.0,133A MOZHAYSKOE H/W. MOSCOW,NaN,NaN,NaN,0.061517,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,3199.0,8083.0,"MOROZOVA P.L., 86 KHabarovsk",NaN,NaN,NaN,NaN,0,0.0,"MOROZOVA P.L., 86 KHabarovsk",...,48.428996,135.103956,NaN,NaN,0.0,5.0,2.0,0.0,NaN,NaN
416,5354.0,5478.0,"D. 15, UL. 78 DOBROVOL'CH KRASNOYARSK",NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
417,3333.0,8083.0,"OKTYABRSKAYA, 105 Tver",NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,2247.0,3185.5,"UL. TELEVIZORNAJA, KRASNOJARSK",NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df = pd.merge(find_gaps, gaps_geocoding,
             how='left',
             left_on='address', 
             right_on='source').drop_duplicates(subset=['id', 'source'])

In [21]:
geocoded_gaps = pd.read_csv('C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\external\\geocoded_data.csv')
geocoded_gaps.drop('Unnamed: 0', axis=1, inplace=True)
geocoded_gaps = geocoded_gaps[['source', 'result', 'geo_lat', 'geo_lon', 'qc_geo']]
geocoded_gaps = geocoded_gaps.drop_duplicates(subset=['source'])
geocoded_gaps = geocoded_gaps[geocoded_gaps['qc_geo'] < 2]
geocoded_gaps.rename(columns={'result': 'address_rus', 'geo_lat': 'lat', 'geo_lon': 'long'}, inplace=True)
geocoded_gaps.drop('qc_geo', axis=1, inplace=True)

In [22]:
geocoded_gaps

,source,address_rus,lat,long
3,113 MIRA AV. TOGLIATTI,"Самарская обл, г Тольятти, ул Мира, влд 113",53.510784,49.444468
6,14/2 BUMAZHNIY AVE MOSCOW,"г Москва, проезд Бумажный, д 14 стр 2",55.789667,37.586913
7,45 YANA RAYNISA ST. MOSCOW,"г Москва, б-р Яна Райниса, д 45",55.849159,37.407386
11,8 NOVATOROV STR. S.-PETERBURG,"г Санкт-Петербург, б-р Новаторов, д 8",59.850555,30.269360
12,199 G.IM.KRASNOYARSKIY RA KRASNOYARSK,"г Красноярск, пр-кт им.газеты ""Красноярский ра...",55.990769,92.887862
...,...,...,...,...
402,"D. 23, MKR. PERVOMAISKII IRKUTSK G","г Иркутск, мкр Первомайский, д 23",52.255019,104.232726
404,"D. 3 A, UL. PRIBOROSTROIT RYBINSK G","Ярославская обл, г Рыбинск, ул Приборостроител...",58.047840,38.781685
408,"D. 25, PR-KT IM MUSY DZHA NABEREZHNYE","Респ Татарстан, г Набережные Челны, пр-кт им М...",55.682255,52.295098
412,"D. 137 A, UL. MICHURINSKA TAMBOV G","г Тамбов, ул Мичуринская, д 137А",52.747608,41.421012


In [23]:
df1 = find_gaps.set_index('address').combine_first(geocoded_gaps.set_index('source'))

In [24]:
df2 = combine_data.set_index('address').combine_first(geocoded_gaps.set_index('source')).reset_index()
df2.rename(columns={'index': 'address'}, inplace=True)

In [25]:
df2[df2['address_rus'].notnull()]

,address,address_rus,atm_group,id,is_train,lat,long,target
0,"30 IYULYA, 8 Krasnoyarsk","улица 30 Июля, 8, Красноярск, Россия, 660021",8083.0,3514.0,1,56.006628,92.830814,0.152559
1,"Bakhturova, 12 Volgograd","улица Бахтурова, 12, Волгоград, Россия",8083.0,4377.0,1,48.528123,44.575958,0.079082
2,"ZAVODSKAYA, 1 Davydovo","Заводская улица, 1, деревня Давыдово, городско...",8083.0,3577.0,1,55.607799,38.860167,0.184321
3,"05 ILSKIJ,MIRA,205 ILSKIJ","улица Мира, 4, посёлок городского типа Ильский...",3185.5,2684.0,0,44.852634,38.589693,NaN
4,1 B.DOROGOMILOVSKAYA ST. MOSCOW,"Большая Дорогомиловская улица, 1, Москва, Росс...",1942.0,628.0,0,55.745549,37.563224,NaN
...,...,...,...,...,...,...,...,...
8759,Zhivopisnaia 136 d.Velednikov,"деревня Веледниково, городской округ Истра, Мо...",32.0,2895.0,1,55.799765,37.116016,0.141469
8760,"im V.I.Lenina pr, 54b Volgograd","проспект имени В.И. Ленина, 54, Волгоград, Рос...",8083.0,4420.0,1,48.723842,44.539532,0.088778
8762,metro Rechnoy Vokzal Novosibirsk,"метро Речной вокзал, Ленинская линия, Новосиби...",8083.0,4239.0,1,55.008573,82.937821,0.092534
8763,"pl. Sovetskaya, 3 N. Novgorod","Советская площадь, 3, Нижний Новгород, Россия,...",3185.5,1926.0,1,56.296199,44.038155,-0.132355


In [26]:
df2[df2['address_rus'].isnull()]

,address,address_rus,atm_group,id,is_train,lat,long,target
40,1 STROITELEY AV. SARATOV,NaN,1942.0,1573.0,0,NaN,NaN,NaN
269,"13, KORP. A, UL. CENTRAL' NARIMANOV G",NaN,5478.0,8174.0,0,NaN,NaN,NaN
301,133A MOZHAYSKOE H/W. MOSCOW,NaN,1942.0,556.0,1,NaN,NaN,0.061517
371,"14B, TRAKTOVAYA ST. SREDNIY",NaN,3185.5,2288.0,1,NaN,NaN,-0.019986
415,16 B.SEMENOVSKAYA MOSCOW,NaN,1942.0,1384.0,1,NaN,NaN,0.095677
...,...,...,...,...,...,...,...,...
8548,UNKNOWN DZERZHINSK G,NaN,5478.0,5530.0,0,NaN,NaN,NaN
8557,USOLIE-SIBIRSKOE 53 KRASNIKH,NaN,1942.0,410.0,0,NaN,NaN,NaN
8569,V/CH. 06987 ENGELS G,NaN,5478.0,7304.0,1,NaN,NaN,-0.010728
8691,"YA KOMSOMOLSK,19A Novo-Aleksandro",NaN,8083.0,3284.0,0,NaN,NaN,NaN


## 5 clean_data

In [27]:
df = pd.read_csv('C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\interim\\combined_data_without_gaps.csv')

In [28]:
df = df[['id', 'atm_group', 'address', 'address_rus', 'lat', 'long', 'target', 'is_train']]

## visualisation

In [29]:
# lat = df['lat']
# lon = df['long']
# atm_group = df['atm_group']

In [30]:
# m = folium.Map(location=[63.391522, 96.328125], zoom_start=5, 
# #                tiles='CartoDB dark_matter'
#               )

In [31]:
# marker_cluster = MarkerCluster().add_to(m)

In [32]:
# for lat, lon, atm_group in zip(lat, lon, atm_group):
#     folium.CircleMarker(
#         location=[lat, lon], 
#         radius=10,
#         popup=str(atm_group),
#         color='grey',
#         fill=True,
#         fill_colour='grey',
#         fill_opacity = 0.9,
# ).add_to(marker_cluster)

In [33]:
# m

In [34]:
# m.save('map1.html')

## reverse geocoding

In [35]:
import json
import numpy as np

In [36]:
with open('C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\external\\reverse_geocoded_data.json', 'r') as f:
    rvrs_geodata = json.load(f)

In [37]:
rvrs_geodata = pd.json_normalize(rvrs_geodata)

In [38]:
#check
rvrs_geodata['geo_city'] = np.where(rvrs_geodata['data.city_with_type'].notnull(),
                                rvrs_geodata['data.city_with_type'],
                                rvrs_geodata['data.settlement_with_type'])

In [39]:
df_with_cities = pd.merge(df,
                          rvrs_geodata[['lat', 'long', 'geo_city']],
                          how='left',
                          on=['lat', 'long'])

In [40]:
pd.options.display.max_rows = 100

In [41]:
df_with_cities[df_with_cities['geo_city'].isnull()]

,id,atm_group,address,address_rus,lat,long,target,is_train,geo_city
57,4713.0,496.5,"1-ST VISHNEVAYA, 17 KURSK","Вишневая улица, СТ Дружба-1, городской округ У...",63.541994,53.852330,-0.053925,1,NaN
525,692.0,1942.0,18/3 ANDROPOVA ST. MOSKVA,"квартал 18/1, 5, сельское поселение Никоновско...",55.343092,38.207945,0.000778,1,NaN
632,437.0,1942.0,2 KRAINYAYA STR. KRASNOYARSK,"автодорога Красноярск-Железногорск, 20 км, Бер...",56.129780,93.332317,0.023905,1,NaN
661,862.0,1942.0,2-5 2-YA REYSOVAYA STR. MOSCOW,"2-я Рейсовая улица, 2к5, посёлок Внуково, Моск...",55.605785,37.287630,0.003121,1,NaN
662,554.0,1942.0,2-5 2-YA REYSOVAYA STR. VNUKOVO,"2-я Рейсовая улица, 2к5, посёлок Внуково, Моск...",55.605785,37.287630,-0.031380,1,NaN
663,555.0,1942.0,2-5 2-YA REYSOVAYA STR. VNUKOVO,"2-я Рейсовая улица, 2к5, посёлок Внуково, Моск...",55.605785,37.287630,-0.036426,1,NaN
945,431.0,1942.0,3 VZLETNAYA STR. KRASNOYARSK,"автодорога Красноярск-Железногорск, 20 км, Бер...",56.129780,93.332317,-0.110161,1,NaN
1137,978.0,1942.0,4-1 22-KM. KIEVSKOE H/W. MOSCOW,"срезка на 5 км, территория Спортивный клуб Ром...",55.749988,37.363538,NaN,0,NaN
1138,977.0,1942.0,4-1 22-KM/ KIEVSKOE H/W. MOSCOW,"срезка на 5 км, территория Спортивный клуб Ром...",55.749988,37.363538,0.060583,1,NaN
1153,1454.0,1942.0,40 LIT A FURSHTATSKAYA SANKT-PETERS,"озеро Лит, Тазовский район, Ямало-Ненецкий авт...",67.502300,80.840713,-0.059737,1,NaN


In [42]:
fill_missing_rvrs_geo[['source', 'result', 'qc_geo', 'geo_city']]

NameError: name 'fill_missing_rvrs_geo' is not defined

In [ ]:
with open('C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\external\\fill_missing_rvrs_geo.json', 'r') as f:
    fill_missing_rvrs_geo = json.load(f)

In [ ]:
fill_missing_rvrs_geo = pd.json_normalize(fill_missing_rvrs_geo)

In [ ]:
fill_missing_rvrs_geo['geo_city'] = np.where(fill_missing_rvrs_geo['city_with_type'].notnull(),
                                fill_missing_rvrs_geo['city_with_type'],
                                fill_missing_rvrs_geo['settlement_with_type'])

In [ ]:
fill_missing_rvrs_geo['geo_city'] = np.where(fill_missing_rvrs_geo['geo_city'].isnull(),
                                fill_missing_rvrs_geo['area_with_type'],
                                fill_missing_rvrs_geo['geo_city'])

In [ ]:
fill_missing_rvrs_geo['geo_city'] = np.where(fill_missing_rvrs_geo['geo_city'].isnull(),
                                fill_missing_rvrs_geo['region_with_type'],
                                fill_missing_rvrs_geo['geo_city'])

In [ ]:
df_with_cities_full = df_with_cities.rename(columns={'city': 'geo_city'}).set_index('address').combine_first(fill_missind_rvrs_geo[['source', 'geo_city']].set_index('source')).reset_index()

In [ ]:
df_with_cities_full[df_with_cities_full['geo_city'].isnull()]

In [ ]:
df_with_cities_full

## Geocode address

In [ ]:
combine_data.iloc[5:10]

In [ ]:
combine_data.drop_duplicates(subset=['address'])

In [ ]:
with open('C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\external\\geocoded_addresses.json', 'r') as f:
    fill_missing_rvrs_geo = json.load(f)

In [ ]:
pd.json_normalize(fill_missing_rvrs_geo)

## geo features

In [ ]:
df.head()

### количество банкоматов по адресу

In [ ]:
df.groupby('address_rus', as_index=False).agg({'id': 'count'})

In [ ]:
df.groupby('address_rus', as_index=False)['id'].agg('count').rename(columns={'id': 'atm_cnt_adrs'})

### количество банкоматов по городу